# Api genel

Spotify Web API (ve dolayısıyla Python kütüphanesi `spotipy`) üzerindeki bu iki akış (flow), aslında **OAuth 2.0** protokolünün farklı kullanım senaryolarıdır. Hangisini seçeceğin tamamen **"Kullanıcının özel verisine ihtiyacım var mı?"** sorusuna vereceğin cevaba bağlıdır.

İşte bu iki akışın temel farkları, kullanım alanları ve kod örnekleri:

-----

### Client Credentials Flow (İstemci Kimlik Bilgileri Akışı)

Bu yöntem, uygulamanın Spotify ile **kullanıcıdan bağımsız** olarak, sunucudan sunucuya (server-to-server) konuşmasıdır.

  * **Temel Amaç:** Halka açık (public) verilere erişmek.
  * **Kullanıcı Girişi:** Gerekmez. (Spotify login ekranı açılmaz).
  * **Erişebileceği Veriler:** Şarkı analizleri (audio features), sanatçı bilgileri, albüm kapakları, herkese açık çalma listeleri.
  * **Erişemeyeceği Veriler:** Bir kullanıcının özel çalma listeleri, kütüphanesine eklediği şarkılar, o an ne dinlediği.
  * **Hız/Kolaylık:** Kurulumu en basit ve en hızlı yöntemdir.

**Ne zaman kullanmalısın?**
Eğer projen sadece veri analizi üzerineyse (örneğin: "Tarkan'ın şarkılarının BPM ve dans edilebilirlik oranlarını analiz etmek istiyorum") bu akışı kullanmalısın.

**Python Örneği:**

```python
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Sadece ID ve Secret yeterlidir
auth_manager = SpotifyClientCredentials(client_id='SENIN_ID',
                                        client_secret='SENIN_SECRET')
sp = spotipy.Spotify(auth_manager=auth_manager)

# Tarkan'ı aratıp verisini çekebilirsin
results = sp.search(q='Tarkan', limit=1)
print(results)
```

-----

### Authorization Code Flow (Yetkilendirme Kodu Akışı)

Bu yöntem, uygulamanın **bir kullanıcı adına** hareket etmesini sağlar.

  * **Temel Amaç:** Kullanıcıya özgü (private) verilere erişmek ve kullanıcı adına işlem yapmak.
  * **Kullanıcı Girişi:** Gereklidir. Tarayıcı açılır, kullanıcı Spotify'a giriş yapar ve "Bu uygulama şunları yapmak istiyor" diyerek izin verir.
  * **Erişebileceği Veriler:** Kullanıcının "Beğenilen Şarkılar"ı, özel çalma listeleri, takip ettiği sanatçılar.
  * **Yetenekleri:** Şarkı değiştirebilir (next/prev), listeye şarkı ekleyebilir, yeni çalma listesi oluşturabilir.
  * **Scope (Kapsam):** Hangi verilere erişeceğini belirtmen gerekir (örn: `user-library-read`, `playlist-modify-public`).
  * **Refresh Token:** Erişim izni (token) süresi dolduğunda (genelde 1 saat), kullanıcıyı tekrar login etmeye gerek kalmadan arkaplanda token'ı yeniler.

**Ne zaman kullanmalısın?**
Eğer kişiselleştirilmiş bir uygulama yapıyorsan (örneğin: "Kullanıcının o an dinlediği şarkıya göre ona benzer şarkılar önerip otomatik olarak çalma listesine ekleyen bir bot") bu akışı kullanmalısın.

**Python Örneği:**

```python
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Redirect URI ve Scope önemlidir
scope = "user-library-read" # Örnek: Kullanıcının kütüphanesini okuma izni

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='SENIN_ID',
                                               client_secret='SENIN_SECRET',
                                               redirect_uri='http://localhost:8888/callback',
                                               scope=scope))

# Kullanıcının son kaydettiği şarkıları çeker
results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])
```

-----

### Karşılaştırma Özeti

| Özellik | Client Credentials Flow | Authorization Code Flow |
| :--- | :--- | :--- |
| **Kullanıcı Etkileşimi** | Yok (Otomatik) | Var (Browser Login gerekir) |
| **Erişim Türü** | Sadece Public (Genel) Veriler | Private (Özel) ve Public Veriler |
| **Yazma Yetkisi** | Yok (Sadece okuma) | Var (Playlist oluşturma, şarkı ekleme vb.) |
| **Kurulum Zorluğu** | Kolay | Orta (Redirect URI ve Token yönetimi) |
| **Token Yenileme** | Basitçe yeni token alınır | Refresh Token mekanizması kullanılır |

### Özet Tavsiye

Eğer amacın **veri bilimi (Data Science)** projeleri yapmak, şarkı özelliklerini analiz etmek veya genel veritabanı oluşturmaksa **Client Credentials Flow** senin için yeterli ve daha az baş ağrıtıcıdır.

Ancak bir **web uygulaması** veya **kullanıcı botu** yazacaksan **Authorization Code Flow** zorunludur.

Hangisi senin şu anki projene daha uygun görünüyor? Yardımcı olmamı ister misin?

# Kurulum ve importlar

In [ ]:
#colab only
!pip -q install spotipy

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials,SpotifyOAuth

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
# #localde, venv içinde pip install dotenv yapılmış olmalı
import os
from dotenv import load_dotenv
load_dotenv()
client_id = os.getenv('Spotipy_Client')
client_secret = os.getenv('Spotipy_Client')

True

In [ ]:
#colabde
# from google.colab import userdata
# client_id = userdata.get('SPOTIPY_CLIENT_ID')
# client_secret = userdata.get('SPOTIPY_CLIENT_SECRET')

In [4]:
import sys
import collections

def total_size(obj, seen=None):
    """Recursively calculates the size of an object in bytes."""
    if seen is None:
        seen = set()

    obj_id = id(obj)
    if obj_id in seen:
        return 0
    seen.add(obj_id)

    size = sys.getsizeof(obj)

    if isinstance(obj, dict):
        size += sum(total_size(k, seen) + total_size(v, seen) for k, v in obj.items())
    elif isinstance(obj, (collections.abc.Iterable, str)) and not isinstance(obj, (bytes, str)):
        size += sum(total_size(item, seen) for item in obj)

    return size

# Örnekler

## Otorizasyonlu

In [5]:
import os

# Check if any environment variable starts with 'COLAB' using any()
if any(key.startswith('COLAB') for key in os.environ.keys()):
    print("There are environment variables starting with 'COLAB':")    
else:
    print("No environment variables starting with 'COLAB' were found.")

No environment variables starting with 'COLAB' were found.


In [6]:
#localde
def getAuth(scope):
    isColab = any(key.startswith('COLAB') for key in os.environ.keys())
    auth = SpotifyOAuth(scope=scope,
                               client_id=client_id,
                               client_secret=client_secret,
                               redirect_uri='https://www.volkanyurtseven.com',
                               open_browser=not isColab)
    return auth


sp_mdf = spotipy.Spotify(auth_manager=getAuth('playlist-modify-private')) #playlilt manipülasyonu için
sp = spotipy.Spotify(auth_manager=getAuth(['playlist-read-private','user-library-read'])) #hem playlisltere hem liked şarkılara erişim için
# sp = spotipy.Spotify(auth_manager=getAuth('user-library-read')) #otorizasyonsuzda sorun çıakrsa bu kısmı çalıştıralım, üsttekini değil

In [7]:
print(sp.current_user())

{'display_name': 'Volkan Yurtseven', 'external_urls': {'spotify': 'https://open.spotify.com/user/11133966425'}, 'followers': {'href': None, 'total': 10}, 'href': 'https://api.spotify.com/v1/users/11133966425', 'id': '11133966425', 'images': [{'height': 300, 'url': 'https://scontent-fra3-2.xx.fbcdn.net/v/t39.30808-1/466921303_10161519827314424_2314629280016839123_n.jpg?stp=cp1_dst-jpg_s320x320_tt6&_nc_cat=104&ccb=1-7&_nc_sid=79bf43&_nc_ohc=9781ILPKzS4Q7kNvwFEPpuN&_nc_oc=AdkI5YdkR6UlZJ1gg6HA2IlDIAUxKQI4ancWRn9Oe2DPI65_KR3TYlxEEMZxBDcH3z6ClWDbprC3g_oQ88opAmq4&_nc_zt=24&_nc_ht=scontent-fra3-2.xx&edm=AP4hL3IEAAAA&_nc_gid=U1iKPGg9xLqbDeMwlZJoUA&_nc_tpa=Q5bMBQHKZd-IBxtawCqjgBapcBG3O9FnHbMbLm20BmRuvdrJUin2YsYpv8tB_Gq2ysxXlwKpckPT&oh=00_Afi9o8zJI6y3TrpmX91SZ0LqQwLGqxn7OEhbptuH75reBQ&oe=6931F124', 'width': 300}, {'height': 64, 'url': 'https://scontent-fra3-2.xx.fbcdn.net/v/t39.30808-1/466921303_10161519827314424_2314629280016839123_n.jpg?stp=cp1_dst-jpg_s50x50_tt6&_nc_cat=104&ccb=1-7&_nc_sid=288

## Otorizasyonsuz

Daha sınırlkı gibi, tüm plasylistler yerine bazıları(?)

In [8]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### user profile

In [9]:
cur_user = sp.current_user()
type(cur_user)

dict

In [10]:
cur_user.keys()

dict_keys(['display_name', 'external_urls', 'followers', 'href', 'id', 'images', 'type', 'uri'])

In [11]:
total_size(cur_user)

3362

In [12]:
cur_user

{'display_name': 'Volkan Yurtseven',
 'external_urls': {'spotify': 'https://open.spotify.com/user/11133966425'},
 'followers': {'href': None, 'total': 10},
 'href': 'https://api.spotify.com/v1/users/11133966425',
 'id': '11133966425',
 'images': [{'height': 300,
   'url': 'https://scontent-fra3-2.xx.fbcdn.net/v/t39.30808-1/466921303_10161519827314424_2314629280016839123_n.jpg?stp=cp1_dst-jpg_s320x320_tt6&_nc_cat=104&ccb=1-7&_nc_sid=79bf43&_nc_ohc=9781ILPKzS4Q7kNvwFEPpuN&_nc_oc=AdkI5YdkR6UlZJ1gg6HA2IlDIAUxKQI4ancWRn9Oe2DPI65_KR3TYlxEEMZxBDcH3z6ClWDbprC3g_oQ88opAmq4&_nc_zt=24&_nc_ht=scontent-fra3-2.xx&edm=AP4hL3IEAAAA&_nc_gid=U1iKPGg9xLqbDeMwlZJoUA&_nc_tpa=Q5bMBQHKZd-IBxtawCqjgBapcBG3O9FnHbMbLm20BmRuvdrJUin2YsYpv8tB_Gq2ysxXlwKpckPT&oh=00_Afi9o8zJI6y3TrpmX91SZ0LqQwLGqxn7OEhbptuH75reBQ&oe=6931F124',
   'width': 300},
  {'height': 64,
   'url': 'https://scontent-fra3-2.xx.fbcdn.net/v/t39.30808-1/466921303_10161519827314424_2314629280016839123_n.jpg?stp=cp1_dst-jpg_s50x50_tt6&_nc_cat=104&ccb

### Playlistler

In [13]:
pls=sp.current_user_playlists()

#başka bir kullanıcı için
# usr='11133966425'
# sp.user_playlists(usr)
pls.keys()
total_size(pls)

dict_keys(['href', 'limit', 'next', 'offset', 'previous', 'total', 'items'])

138303

In [14]:
pls['total']

38

In [15]:
pls["items"][0]

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/1XXap8kxPYdZpI2J85K2Or'},
 'href': 'https://api.spotify.com/v1/playlists/1XXap8kxPYdZpI2J85K2Or',
 'id': '1XXap8kxPYdZpI2J85K2Or',
 'images': [{'height': 640,
   'url': 'https://mosaic.scdn.co/640/ab67616d00001e02744f1e5385fabb74cf6dadb4ab67616d00001e028f375bff308d754e6f960dcdab67616d00001e0291963cc221dc998045e40354ab67616d00001e02eb3af9c8a49d140f48dd79e9',
   'width': 640},
  {'height': 300,
   'url': 'https://mosaic.scdn.co/300/ab67616d00001e02744f1e5385fabb74cf6dadb4ab67616d00001e028f375bff308d754e6f960dcdab67616d00001e0291963cc221dc998045e40354ab67616d00001e02eb3af9c8a49d140f48dd79e9',
   'width': 300},
  {'height': 60,
   'url': 'https://mosaic.scdn.co/60/ab67616d00001e02744f1e5385fabb74cf6dadb4ab67616d00001e028f375bff308d754e6f960dcdab67616d00001e0291963cc221dc998045e40354ab67616d00001e02eb3af9c8a49d140f48dd79e9',
   'width': 60}],
 'name': 'Efso klasik ',
 'owner': {'dis

In [16]:
# @title
[x["name"] for x in pls["items"]]

['Efso klasik ',
 'Havuz',
 'Italian Café Jazz',
 'Underrated songs',
 'Değeri bilinmeyen şarkılar',
 'Feel good',
 'Bossa Nova Covers Popular Songs + 200 Hits',
 'Bosa Nova Covers 2025 🌴 bossa nova music',
 'Atatürk’ün Sevdiği Türküler ',
 'Kitap',
 'Discovered by Musixmatch',
 'Champion_0_Mixed',
 'Champion_3_Energic',
 'Champion_2_Instrumental',
 'Champion_1_Slow',
 'Voted_0_Instrumental',
 'Voted_3_Medium',
 'Voted_2_Slow',
 'Voted_1_Energic',
 'SoundHound',
 'Amelie Soundtracks',
 'Lofi Fruits Music - lofi hip hop beats to study, relax & sleep to',
 'Study Jazz',
 'Özgün coşku',
 'Özgün soft',
 "Radyo'dan Beğenilenler",
 'Chillout - Old Classic',
 'Chillout - Instrumental',
 'Chillout - MorningCoffee',
 'Chillout - New Age',
 'Slow pure',
 'Slowish PopRock',
 'Vivid - Classic',
 'Vivid- Instrumental',
 'Vivid- PopRock',
 'Vivid- RapHiphop',
 'Vivid - Soft',
 'TürküDeyiş']

In [17]:
#playliste kapsamlı obje oalrak erişim
playlist1 = sp.playlist(pls["items"][0]["id"])
playlist1

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/1XXap8kxPYdZpI2J85K2Or'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/1XXap8kxPYdZpI2J85K2Or?additional_types=track',
 'id': '1XXap8kxPYdZpI2J85K2Or',
 'images': [{'height': 640,
   'url': 'https://mosaic.scdn.co/640/ab67616d00001e02744f1e5385fabb74cf6dadb4ab67616d00001e028f375bff308d754e6f960dcdab67616d00001e0291963cc221dc998045e40354ab67616d00001e02eb3af9c8a49d140f48dd79e9',
   'width': 640},
  {'height': 300,
   'url': 'https://mosaic.scdn.co/300/ab67616d00001e02744f1e5385fabb74cf6dadb4ab67616d00001e028f375bff308d754e6f960dcdab67616d00001e0291963cc221dc998045e40354ab67616d00001e02eb3af9c8a49d140f48dd79e9',
   'width': 300},
  {'height': 60,
   'url': 'https://mosaic.scdn.co/60/ab67616d00001e02744f1e5385fabb74cf6dadb4ab67616d00001e028f375bff308d754e6f960dcdab67616d00001e0291963cc221dc998045e40354ab67616d00001e02eb3af9c8a49d140f48dd

In [18]:
playlist1.keys()

dict_keys(['collaborative', 'description', 'external_urls', 'followers', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

In [19]:
playlist1["tracks"].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [20]:
songs = playlist1["tracks"]['items']
type(songs)
songs

list

[{'added_at': '2025-08-31T10:33:21Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/11133966425'},
   'href': 'https://api.spotify.com/v1/users/11133966425',
   'id': '11133966425',
   'type': 'user',
   'uri': 'spotify:user:11133966425'},
  'is_local': False,
  'primary_color': None,
  'track': {'preview_url': None,
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
    'LU',
    'MY',
    'MT',
    'MX',
    'NL',
    'NZ',
    'NI',
    'NO',
    'PA',
    'PY',
    'PE',
    'PH',
    'PL',
    'PT',
    'SG',
    'SK',
    'ES',
    'SE',
    'CH',
    'TW',
    'TR',
    'UY',
    'US',
    'GB',
    'AD',
    'LI',
    'MC',
    'ID',
    'JP',
    'TH',
    'VN',
    'RO',
 

In [21]:
[x['track']["name"] for x in songs]

['Schubert: Serenada',
 'Prince Igor : Polovtsian Dances I. Introduzione, Andantino',
 'Air from Orchestral Suite No. 3',
 'Oboe Concerto in D Minor: II. Adagio',
 'Oboe Concerto in D Minor, S. Z799: II. Adagio',
 'Concerto in D minor BWV 974: II. Adagio (after the Oboe concerto by Alessandro Marcello, transcription by J. S. Bach)',
 'Handel: Suite No. 11 in D Minor, HWV 437: III. Sarabande - Remastered 2022',
 'Concerto in D Minor, BWV 974: II. Adagio (Arr. for Cello and Piano by Mischa Maisky)',
 'Suite No. 3 in D Major, BWV 1068: II. Air',
 'Ständchen in D Minor (After Schubert), S. 560',
 'Chopin Nocturne Opus 9 No 1',
 'Saint-Saëns: Le carnaval des animaux, R 125: XIII. Le cygne',
 'Rodrigo: Concierto de Aranjuez: II. Adagio (Excerpt)',
 'Jeux Interdits: Spanish Romance (Arr. for Guitar and Orchestra)',
 'Nocturnes, Op. 9: No. 1 in B-Flat Minor',
 "Mariage d'Amour",
 'Nocturne in C Sharp Minor',
 'Canon in D Major',
 'Symphony No. 7 in A Major, Op. 92: II. Allegretto',
 'Lakmé: Fl

### saved tracks(beğenilen şarkılar)

In [22]:
liked_songs = sp.current_user_saved_tracks() #defaulat 20 adet
type(liked_songs),total_size(liked_songs)

(dict, 515112)

In [23]:
len(liked_songs)
liked_songs.keys()

7

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [24]:
len(liked_songs['items'])

20

In [25]:
liked_songs['items'][0]

{'added_at': '2025-11-24T10:24:36Z',
 'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4tZwfgrHOc3mvqYlEYSvVi'},
     'href': 'https://api.spotify.com/v1/artists/4tZwfgrHOc3mvqYlEYSvVi',
     'id': '4tZwfgrHOc3mvqYlEYSvVi',
     'name': 'Daft Punk',
     'type': 'artist',
     'uri': 'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
    'LU',
    'MY',
    'MT',
    'MX',
    'NL',
    'NZ',
    'NI',
    'NO',
    'PA',
    'PY',
    'PE',
    'PH',
    'PL',
    'PT',
    'SG',
    'SK',
    'ES',
    'SE',
    'CH',
    'TW',
    'TR',
    'UY',
    'US',
    'GB',
    'AD',
    'LI',
    'MC',

In [26]:
# sanatçı bilgsi
liked_songs['items'][0]["track"]["album"]["artists"][0]["name"]

'Daft Punk'

In [27]:
#şarkı ve albüm bilgisi
liked_songs['items'][0]["track"]["album"]["name"], liked_songs['items'][0]["track"]["name"]

('Random Access Memories (10th Anniversary Edition)',
 'Touch (2021 Epilogue) [feat. Paul Williams]')

In [28]:
for idx, item in enumerate(liked_songs['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 Daft Punk  –  Touch (2021 Epilogue) [feat. Paul Williams]
1 Daft Punk  –  Touch (feat. Paul Williams)
2 Chezile  –  Beanie
3 Daniel Ferri  –  Instant Crush
4 Daft Punk  –  Get Lucky (feat. Pharrell Williams and Nile Rodgers)
5 Rondò Veneziano  –  La Serenissima
6 Daft Punk  –  Instant Crush (feat. Julian Casablancas)
7 The Weeknd  –  Save Your Tears (Remix) (with Ariana Grande) - Bonus Track
8 The Weeknd  –  Blinding Lights
9 The Weeknd  –  In Your Eyes
10 Cj Borika  –  Veridis Quo
11 Original Orn  –  Veridis Quo
12 Daft Punk  –  Veridis Quo
13 Josh Vietti  –  Canon In D
14 Lady Gaga  –  Die With A Smile
15 The White Buffalo  –  House of the Rising Sun (with The Forest Rangers) - Season 4 Finale Version
16 The Animals  –  House of the Rising Sun
17 40 Fingers  –  Last of the Mohicans (Live)
18 Damiano David  –  Talk to Me (feat. Tyla, Nile Rodgers)
19 Led Zeppelin  –  Stairway to Heaven - Remaster


# Görevler

## Playlsitlerdeki distinc şarkı listesi

In [29]:
def GetUniqueSongsInPlaylists(sp):
    playlists = sp.current_user_playlists()["items"]
    songlist=[]

    for playlist in playlists:
        pl=sp.playlist(playlist["id"])
        songs=pl["tracks"]['items']
        for i in range(len(songs)):
            song=songs[i]['track']
            if song is None: #silinmişlik kontrolü
              print(f"None alert: {songs[i]}")
            else:
              if song["id"] not in [x for x,_,_ in songlist]: #duplike kontrolü
                  songlist.append((song["id"],song["name"],pl["name"]))
              else:
                  print(f"{playlist["name"]} içindeki {song['id'],song['name']} şarkısı zaten başka bir playlistten daha önce eklenmiş")

    return songlist

In [30]:
playlist_songs=GetUniqueSongsInPlaylists(sp)
len(playlist_songs)

Italian Café Jazz içindeki ('4pil6t7EZUCGF2PlxMiBed', 'Snapback') şarkısı zaten başka bir playlistten daha önce eklenmiş
Feel good içindeki ('27enWlEQc6sWiyMiWW9E8S', 'Mevsim Bahar') şarkısı zaten başka bir playlistten daha önce eklenmiş
Feel good içindeki ('4C037NIJEXLzcOOZNxLJ59', 'New Light') şarkısı zaten başka bir playlistten daha önce eklenmiş
Feel good içindeki ('41f8HyuxASzuEcEs01x10O', 'New Light') şarkısı zaten başka bir playlistten daha önce eklenmiş
Feel good içindeki ('3WkLwto0IaaxjFeetMgdRJ', 'Birdenbire') şarkısı zaten başka bir playlistten daha önce eklenmiş
Bossa Nova Covers Popular Songs + 200 Hits içindeki ('2B0xND8Y5274NEEF5hZtPQ', 'Words - Blue System Remix') şarkısı zaten başka bir playlistten daha önce eklenmiş
Kitap içindeki ('0IlmPZGIQb1j1eOV0q3RZ9', 'Nocturnes, Op. 9: No. 1 in B-Flat Minor') şarkısı zaten başka bir playlistten daha önce eklenmiş
Discovered by Musixmatch içindeki ('2xV3JIIeKgIvsJRLFgReCU', 'Pink Venom') şarkısı zaten başka bir playlistten daha 

1331

In [31]:
playlist_songs[0]

('124VZO1gdikABHpdFE9x27', 'Schubert: Serenada', 'Efso klasik ')

In [32]:
from collections import Counter
c=Counter([x[2] for x in playlist_songs])
c.most_common(3)

[('Bosa Nova Covers 2025 🌴 bossa nova music', 100),
 ('Atatürk’ün Sevdiği Türküler ', 100),
 ('Champion_1_Slow', 100)]

In [33]:
#playslitlerdeki gerçek şarkı adedi böyle bulunur, üstteki yöntemde bi şarkının ilk geçtiği playlist alındı
pls=sp.current_user_playlists()
playlists_names = [x["name"] for x in pls["items"]]
playlists_counts = [x["tracks"]["total"] for x in pls["items"]]
dict(zip(playlists_names,playlists_counts))
print("\nTotal number", sum(playlists_counts))

{'Efso klasik ': 24,
 'Havuz': 19,
 'Italian Café Jazz': 56,
 'Underrated songs': 2,
 'Değeri bilinmeyen şarkılar': 2,
 'Feel good': 13,
 'Bossa Nova Covers Popular Songs + 200 Hits': 300,
 'Bosa Nova Covers 2025 🌴 bossa nova music': 338,
 'Atatürk’ün Sevdiği Türküler ': 105,
 'Kitap': 25,
 'Discovered by Musixmatch': 15,
 'Champion_0_Mixed': 80,
 'Champion_3_Energic': 225,
 'Champion_2_Instrumental': 105,
 'Champion_1_Slow': 239,
 'Voted_0_Instrumental': 139,
 'Voted_3_Medium': 163,
 'Voted_2_Slow': 163,
 'Voted_1_Energic': 184,
 'SoundHound': 8,
 'Amelie Soundtracks': 113,
 'Lofi Fruits Music - lofi hip hop beats to study, relax & sleep to': 250,
 'Study Jazz': 131,
 'Özgün coşku': 16,
 'Özgün soft': 27,
 "Radyo'dan Beğenilenler": 14,
 'Chillout - Old Classic': 9,
 'Chillout - Instrumental': 44,
 'Chillout - MorningCoffee': 26,
 'Chillout - New Age': 5,
 'Slow pure': 34,
 'Slowish PopRock': 41,
 'Vivid - Classic': 6,
 'Vivid- Instrumental': 8,
 'Vivid- PopRock': 52,
 'Vivid- RapHipho


Total number 3130


## Tüm beğenilen listesini alma

In [34]:
likedsongs=[]
for i in range(878//50+1):
    temp=sp.current_user_saved_tracks(limit=50,offset=i*50)
    likedsongs.extend((x["track"]["id"],x["track"]["name"],x["track"]["artists"][0]["name"]) for x in temp["items"])
likedsongs[0]
len(likedsongs)

('7oMbGRDGVDIb6ZeVcq7LfW',
 'Touch (2021 Epilogue) [feat. Paul Williams]',
 'Daft Punk')

878

In [35]:
#while ile yapılışı
likedsongs = []
offset = 0
limit = 50

while True:
    temp = sp.current_user_saved_tracks(limit=limit, offset=offset)
    # Add the track information to the list
    likedsongs.extend((x["track"]["id"], x["track"]["name"], x["track"]["artists"][0]["name"]) for x in temp["items"])

    # If there are no more pages, break the loop
    if temp["next"] is None:
        break

    # Increment the offset for the next iteration
    offset += limit

print(f"Number of liked songs fetched with while loop: {len(likedsongs)}")
print("First 3 liked songs:")
print(likedsongs[:3])

Number of liked songs fetched with while loop: 878
First 3 liked songs:
[('7oMbGRDGVDIb6ZeVcq7LfW', 'Touch (2021 Epilogue) [feat. Paul Williams]', 'Daft Punk'), ('7oaEjLP2dTJLJsITbAxTOz', 'Touch (feat. Paul Williams)', 'Daft Punk'), ('5e0b9LgOfi3aJSKXFcOWRe', 'Beanie', 'Chezile')]


## songs that are in the library but not added to playlists

In [36]:
plss=[s for s,_,_ in playlist_songs]
ls=[s for s,_,_ in likedsongs]
diff1=set(ls).difference(set(plss))
len(diff1)

288

tersine bakalım

In [37]:
diff2=set(plss).difference(set(ls))
len(diff2)

741